In [1]:
import json
import re

import pandas as pd
from sklearn import preprocessing
import torch
device = torch.device("cuda:0")
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torchvision.models as models

/home/ming0531/miniconda3/envs/drct/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_type = "train"
# temporal
with open("/ssd5/ming/CPDN/dataset/train_allmetadata_json/train_temporalspatial_information.json" % (data_type, data_type)) as f:
    train_ts = json.load(f)
    vuid_arr = []
    time_arr = []
    lon_arr, lat_arr, geoacc_arr = [], [], []
    ts_dict = {}
    # 对时间排序
    for k, v in enumerate(train_ts):
        vuid = v['Uid'] + '/' + v['Pid']
        vuid_arr.append(vuid)
        time_arr.append(v['Postdate'])
        if v['Geoaccuracy'] == '0':
            lon_arr.append(0)
            lat_arr.append(0)
            geoacc_arr.append(0)
        else:
            lon_arr.append(v['Longitude'])
            lat_arr.append(v['Latitude'])
            geoacc_arr.append(v['Geoaccuracy'])
    id_arr = list(range(0, len(time_arr)))
f.close()


In [16]:

with open("/ssd5/ming/CPDN/dataset/%s_allmetadata_json/%s_category.json" % (data_type, data_type)) as f:
    train_cat = json.load(f)
    cat_arr, subcat_arr, concept_arr = [], [], []
    cat_sort, subcat_sort, concept_sort = [], [], []
    for v in train_cat:
        cat_arr.append(v['Category'])
        subcat_arr.append(v['Subcategory'])
        concept_arr.append(v['Concept'])
    for id in id_sort:
        cat_sort.append(cat_arr[id])
        subcat_sort.append(subcat_arr[id])
        concept_sort.append(concept_arr[id])
f.close()

In [17]:

with open("/ssd5/ming/CPDN/dataset/%s_allmetadata_json/%s_text.json" % (data_type, data_type)) as f:
    train_text = json.load(f)
    text_arr, text_sort = [], []
    tags_arr, tags_sort = [], []
    for v in train_text:
        text_arr.append(v['Title'])
        tags_arr.append(v['Alltags'])
    for id in id_sort:
        text_sort.append(text_arr[id])
        tags_sort.append(tags_arr[id])
f.close()

In [7]:
len(path_arr)

486194

In [ ]:

tensor([     0,    139, 218646, 438926, 444739, 444740, 448089, 452556])

In [46]:
path_arr[438926]

'architect4'

In [47]:
path_arr[0]

'luisdrayton'

In [ ]:
for i in path_arr:
    print(i)

In [44]:
data_type='test'

In [45]:
with open("/ssd5/ming/CPDN/dataset/%s_allmetadata_json/%s_additional_information.json" % (data_type, data_type)) as f:
    train_add = json.load(f)
    #public_arr, public_sort, path_arr, path_sort = [], [], [], []
    for v in train_add:
        vuid = v['Uid'] + '/' + v['Pid']
        public_arr.append(v['Ispublic'])
        path_arr.append(v['Pathalias'])
    for id in id_sort:
        public_sort.append(public_arr[id])
        path_sort.append(path_arr[id])
f.close()


In [19]:
if data_type == "train":
    with open("/ssd5/ming/CPDN/dataset/train_allmetadata_json/train_label.txt") as f:
        label_arr, label_sort = [], []
        for ls in f.readlines():
            l = ls.strip().split(' ')[0]
            label_arr.append(float(l))
        for id in id_sort:
            label_sort.append(label_arr[id])
    f.close()

In [20]:
# user
fill_loc = "0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0"
with open("/ssd5/ming/CPDN/dataset/%s_allmetadata_json/%s_user_data.json" % (data_type, data_type)) as f:
    train_user = json.load(f)
    ispro_arr, ispro_sort, pcount_arr, pcount_sort, canpro_arr, canpro_sort = [], [], [], [], [], []
    loc_arr, loc_sort = [], []
    tzid_arr, tzid_sort, tzoffset_arr, tzoffset_sort = [], [], [], []
    loc_str = ''
    for v in train_user:
        ispro_arr.append(v['ispro'])
        pcount_arr.append(v['photo_count'])
        canpro_arr.append(v['canbuypro'])
        tzid_arr.append(v['timezone_timezone_id'])
        tzoffset_arr.append(v['timezone_offset'])
        if v['location_description'] == "\n":
            loc_str = fill_loc
        else:
            loc_str = v['location_description']
        loc_str_list = loc_str.split(',')
        loc_list = []
        for i in loc_str_list:
            loc_list.append(float(i))
        loc_arr.append(loc_list)
    #print(vuid_arr[165118], vuid_arr[123131])
    #print(pcount_arr[165118], pcount_arr[123131])
    #print(ispro_arr[165118], ispro_arr[123131])
    for id in id_sort:
        ispro_sort.append(ispro_arr[id])
        pcount_sort.append(pcount_arr[id])
        canpro_sort.append(canpro_arr[id])
        tzid_sort.append(tzid_arr[id])
        tzoffset_sort.append(tzoffset_arr[id])
        loc_sort.append((loc_arr[id]))
pcount_sort = list(preprocessing.minmax_scale(pcount_sort))
f.close()
np.save('/ssd5/ming/CPDN/dataset/features/test_location.npy', np.asarray(loc_sort))

In [21]:
if data_type == "train":
    data = zip(*[id_sort, vuid_sort, time_sort, cat_sort, subcat_sort, concept_sort, text_sort, tags_sort, public_sort, ispro_sort, pcount_sort , canpro_sort, tzid_sort\
        ,tzoffset_sort, public_sort, path_sort, lon_sort, lat_sort, geoacc_sort,  label_sort])
    dataframe = pd.DataFrame(data, columns=['id', 'vuid', 'time', 'category', 'subcategory', 'concept', 'text', 'tags', 'ispublic', 'ispro', 'pcount', 'canbuypro',\
        'tzid','tzoffset_sort', 'public', 'pathalias', 'longitude', 'latitude', 'geoacc', 'label'])
    dataframe.to_csv('/ssd5/ming/CPDN/dataset/train_data_0.csv')

else:
    data = zip(*[id_sort, vuid_sort, time_sort, cat_sort, subcat_sort, concept_sort, text_sort, tags_sort, public_sort, ispro_sort, pcount_sort , canpro_sort, tzid_sort\
        ,tzoffset_sort, public_sort, path_sort,  lon_sort, lat_sort, geoacc_sort])
    dataframe = pd.DataFrame(data, columns=['id', 'vuid', 'time', 'category', 'subcategory', 'concept', 'text', 'tags', 'ispublic', 'ispro', 'pcount', 'canbuypro',\
        'tzid','tzoffset_sort', 'public', 'pathalias', 'longitude', 'latitude', 'geoacc'])
    dataframe.to_csv('/ssd5/ming/CPDN/dataset/test_data_0.csv')


if data_type == "train":
    data = zip(*[id_arr, vuid_arr, time_arr, cat_arr, subcat_arr, concept_arr, text_arr, tags_arr, public_arr, ispro_arr, pcount_arr , canpro_arr, tzid_arr\
        ,tzoffset_arr, public_arr, path_arr, lon_arr, lat_arr, geoacc_arr, label_arr])
    dataframe = pd.DataFrame(data, columns=['id', 'vuid', 'time', 'category', 'subcategory', 'concept', 'text', 'tags', 'ispublic', 'ispro', 'pcount', 'canbuypro',\
        'tzid','tzoffset_sort', 'public', 'pathalias','longitude', 'latitude', 'geoacc', 'label'])
    dataframe.to_csv('/ssd5/ming/CPDN/dataset/train_data_1.csv')
else:
    data = zip(*[id_arr, vuid_arr, time_arr, cat_arr, subcat_arr, concept_arr, text_arr, tags_arr, public_arr, ispro_arr, pcount_arr , canpro_arr, tzid_arr\
        ,tzoffset_arr, public_arr, path_arr, lon_arr, lat_arr, geoacc_arr])
    dataframe = pd.DataFrame(data, columns=['id', 'vuid', 'time', 'category', 'subcategory', 'concept', 'text', 'tags', 'ispublic', 'ispro', 'pcount', 'canbuypro',\
        'tzid','tzoffset_sort', 'public', 'pathalias', 'longitude', 'latitude', 'geoacc'])
    dataframe.to_csv('/ssd5/ming/CPDN/dataset/test_data_1.csv')

In [ ]:
image_features = []
text_features = []
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
for name, param in model.named_parameters():
    print(name, param.size())
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [23]:
batch = 512
null=0
a = int(len(vuid_arr) / batch)
b = len(vuid_arr) - a * batch

print(a, b)

596 461


In [24]:
352*512+357

180581

In [25]:
sentence = str("This is a social media post, the category of this post is" + cat_arr[0] + ", the subcategory is " + subcat_arr[0]\
         + ", the concept is " + concept_arr[0] + ", the title is " + text_arr[0] + ", and the hashtags are" + tags_arr[0] +".")
print(sentence)

This is a social media post, the category of this post isFashion, the subcategory is Fashion, the concept is glam, the title is Luis Drayton - Edinburgh shoot #6, and the hashtags arerock punk transgender tranny electronicmusic electro glam electronica luisdrayton fusionrecords thefusionnetwork lmwcphotography.


In [30]:
import cv2

In [1]:
batch = 256
null=0
a = int(len(vuid_arr) / batch)
b = len(vuid_arr) - a * batch
for j in range(0, a):
    #print(j)
    images = []
    texts = []
    for i in range(j * batch, (j + 1) * batch):
        #text = ' '.join(tags_arr[i])
        text = str("This is a social media post, the category of this post is" + cat_arr[0] + ", the subcategory is " + subcat_arr[0]\
         + ", the concept is " + concept_arr[0] + ", the title is " + text_arr[0] + ", and the hashtags are" + tags_arr[0] +".")
        image_path = "/ssd6/ming/eva_embedding/train/" + vuid_arr[i] + '.jpg'
        try:
            image = Image.open(image_path)
        except:
            print(null,image_path)
            null=null+1
            image = Image.open('/ssd5/ming/CPDN/blank.png')
            #print(null,image_path)
        images.append(image)
        texts.append(text)
    inputs = processor(text=texts, images=images, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    image_feature = outputs.vision_model_output.pooler_output
    text_feature = outputs.text_model_output.pooler_output
    print(image_feature.shape, text_feature.shape)
    image_features.append(image_feature.cpu().numpy())
    text_features.append(text_feature.cpu().numpy())
#82m54.8s

NameError: name 'vuid_arr' is not defined

In [ ]:
images = []
texts = []
for i in range(a * batch, a * batch + b):
    print(i)
    text = str("This is a social media post, the category of this post is" + cat_arr[i] + ", the subcategory is " + subcat_arr[i]\
         + ", the concept is " + concept_arr[i] + ", the title is " + text_arr[i] + ", and the hashtags are" + tags_arr[i] +".")

    image_path = "/ssd6/ming/eva_embedding/test/" + vuid_arr[i] + '.jpg'
    try:
        image = Image.open(image_path)
    except:
        null=null+1
        image = Image.open('/ssd5/ming/CPDN/blank.png')
        print(null)
    images.append(image)
    texts.append(text)
inputs = processor(text=texts, images=images, return_tensors="pt", truncation=True, padding=True).to(device)
with torch.no_grad():
    outputs = model(**inputs)
image_feature = outputs.vision_model_output.pooler_output
text_feature = outputs.text_model_output.pooler_output
image_features.append(image_feature.cpu().numpy())
text_features.append(text_feature.cpu().numpy())
#5.7s

In [ ]:
a*512+b

305613

In [ ]:
n = np.vstack(text_features)
np.save("/ssd5/ming/CPDN/dataset/features/test_text_clip.npy", n)
m = np.vstack(image_features)
np.save("/ssd5/ming/CPDN/dataset/features/test_image_clip.npy", m)

In [ ]:
with open("/ssd5/ming/CPDN/dataset/%s_allmetadata_json/%s_additional_information.json" % (data_type, data_type)) as f:
    train_add = json.load(f)
    public_arr, public_sort, path_arr, path_sort = [], [], [], []
    for v in train_add:
        vuid = v['Uid'] + '/' + v['Pid']
        public_arr.append(v['Ispublic'])
        path_arr.append(v['Pathalias'])
    for id in id_sort:
        public_sort.append(public_arr[id])
        path_sort.append(path_arr[id])
f.close()
additional_data = pd.read_csv("/ssd5/ming/CPDN/dataset/clean_user_additional_20240424.csv")

In [ ]:
additional_data.columns

Index(['name', 'testimonials_nums', 'photosof_nums', 'photoCount',
       'person_models_length', 'totalViews', 'totalTags', 'totalGeotagged',
       'totalFaves', 'totalInGroup', 'followerCount', 'followingCount'],
      dtype='object')

In [ ]:
user_info_dict = additional_data.set_index('name').T.to_dict('list')

# Creating a result dictionary that will contain the usernames from path_arr
# and their corresponding additional information if found in the DataFrame
additional_dict = {username: user_info_dict.get(username) for username in path_arr}
additional_dict

In [ ]:
with open("/ssd5/ming/CPDN/dataset/test_additional.json", "w") as f:
    json.dump(additional_dict, f)

In [ ]:
testimonials_nums, photosof_nums, photoCount, person_models_length, totalViews, totalTags = [], [], [], [], [], []
totalGeotagged, totalFaves, totalInGroup, followerCount, followingCount = [], [], [], [], []
for id in path_arr:
    try:
        testimonials_nums.append(additional_dict[id][0])
    except:
        testimonials_nums.append(0)
for id in path_arr:
    try:
        photosof_nums.append(additional_dict[id][1])
    except:
        photosof_nums.append(0)
for id in path_arr:
    try:
        photoCount.append(additional_dict[id][2])
    except:
        photoCount.append(0)
for id in path_arr:
    try:
        person_models_length.append(additional_dict[id][3])
    except:
        person_models_length.append(0)

for id in path_arr:
    try:
        totalViews.append(additional_dict[id][4])
    except:
        totalViews.append(0)
for id in path_arr:
    try:
        totalTags.append(additional_dict[id][5])
    except:
        totalTags.append(0)
for id in path_arr:
    try:
        totalGeotagged.append(additional_dict[id][6])
    except:
        totalGeotagged.append(0)
for id in path_arr:
    try:
        totalFaves.append(additional_dict[id][7])
    except:
        totalFaves.append(0)  
for id in path_arr:
    try:
        totalInGroup.append(additional_dict[id][8])
    except:
        totalInGroup.append(0)
for id in path_arr:
    try:
        followerCount.append(additional_dict[id][9])
    except:
        followerCount.append(0)
        
for id in path_arr:
    try:
        followingCount.append(additional_dict[id][10])
    except:
        followingCount.append(0)

In [2]:
data_type='test'
with open("/ssd5/ming/CPDN/dataset2/%s_allmetadata_json/%s_user_data.json" % (data_type, data_type)) as f:
    train_user = json.load(f)
pattern = r'\b(?:http://|www\.)\S+?\b'

phone_pattern = r'\(?\d{1,3}\)?[\s-]?\d{1,4}[\s-]?\d{1,4}[\s-]?\d{1,4}'

# URL模式 - 匹配典型的网址
url_pattern = r'\b(?:https?://)?\w+\.com\b(?:[\/\w-]*)*'

# 将电话号码和URL的正则表达式合并
combined_pattern = f'{phone_pattern}|{url_pattern}'

#
# 使用 re.sub() 移除这些匹配的文本
#for key, value in enumerate(train_user['user_description']):
#    print(key, re.sub(combined_pattern, '', re.sub(pattern, '', re.sub(r'<[^>]+>', '', train_user['user_description'][str(key)]))).replace('?','') if train_user['user_description'][str(key)] else 'Unknown')

In [3]:
user_description = []
# 使用 re.sub() 移除这些匹配的文本
for key, value in enumerate(train_user['user_description']):
    user_description.append(re.sub(combined_pattern, '', re.sub(pattern, '', re.sub(r'<[^>]+>', '', train_user['user_description'][str(key)]))).replace('?','').replace('*','') if train_user['user_description'][str(key)] else 'Unknown')

In [ ]:
user_description

In [ ]:
from sentence_transformers import SentenceTransformer
import pickle
import pandas as pd
import numpy as np
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print('read!')
embeddings = embedder.encode(user_description)
print(embeddings)
print(len(embeddings[0]))
#35m 28.0s for training data
#28m 21.7s for testing data

In [7]:
n = np.vstack(embeddings)
len(n)

180581

In [8]:
na = np.vstack(embeddings)
np.save("/ssd5/ming/CPDN/dataset/features/test_user_description.npy", na)